## Prepare data

뉴스기사에서 정치인 혹은 직책명과 연예인 이름을 찾는 window classification 기반 모델을 만들어 봅니다. 앞서 Logistic regression 을 이용할 때에는 sparse vector 로 context words 를 입력하였습니다. 이 부분을 Word2Vec 의 벡터로 변환하여 입력합니다.

토크나이징을 위하여 명사 추출을 한 뒤, L-Tokenizer 로 명사 부분을 어절에서 잘라내었습니다.

In [ ]:
import config
from navernews_10days import get_news_paths
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from soynlp.utils import DoublespaceLineCorpus

path = get_news_paths(date='2016-10-20')
sents = list(DoublespaceLineCorpus(path, iter_sent=True))

noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(sents)
noun_score = {noun:score.score for noun, score in nouns.items()}

tokenizer = LTokenizer(scores=noun_score)
sents = [tokenizer.tokenize(sent) for sent in sents]

Word2Vec 을 이용하여 명사와 그 외의 어절들의 word vector 를 학습합니다.

In [ ]:
import numpy as np
from gensim.models import Word2Vec

word2vec = Word2Vec(sents)

wv = word2vec.wv.vectors

Word2Vec 은 infrequent words 에 대해서는 단어 벡터가 잘 학습되지 않습니다. 35000 번째 단어의 빈도수가 10 이니 이 단어까지만 이용합니다. 학습이 잘 된 단어 벡터에 대한 기준은 이 블로그를 참고하시기 바랍니다.

https://lovit.github.io/nlp/representation/2018/12/05/min_count_of_word2vec/

In [ ]:
str(word2vec.wv.vocab[word2vec.wv.index2word[35000]])

알려지지 않은 단어에 대해서는 모두 zero vector 를 할당합니다. 35001 번째 단어는 unknown 입니다.

In [ ]:
wv_ = np.vstack([wv[:35000], np.zeros((1, wv.shape[1]))])
vocab_to_idx = {vocab:idx for idx, vocab in enumerate(word2vec.wv.index2word[:35000])}

Word2Vec 모델의 유사어 검색 기능을 이용하여 seed words 로 이용할 단어를 선정합니다.

In [ ]:
seed_words = {w for w, _ in word2vec.wv.most_similar('김무성', topn=50)}
seed_words.update({w for w, _ in word2vec.wv.most_similar('노무현', topn=50)})
seed_words = set(
    '''4선 강석진 강석호 과학기술계 권석창 권은희 김관영 김기선 김대중 김동철
    김만복 김명연 김성식 김영호 김용국 김정우 김정일 남재준 노무현정부 대구시의회
    대변인 대북인권 미셰우 민화 박근혜 박덕흠 박정희 박주민 박주선 박찬대 백종천
    버락 부시 비대위 비서실장 상근부회장 서청원 송원영 송하진 아베 아키노 안보실장
    연설기획비서관 오바마 유동수 이낙연 이명박 이장우 이재정 이정훈 이종걸 이종배'''.split())
#    연설기획비서관 오바마 유동수 이낙연 이명박 이장우 이재정 이정훈 이종걸 이종배
#    인수위원회 임종성 전두환 전북도지사 정우택 지우마 진선미 최연혜 충북경제자유구역청
#    충북도의회 테메르 통일부장관 호세프 황광 황주홍 후세인'''.split())

`seed_words` 가 하나라도 포함된 문장만 학습에 이용합니다. 그 외의 문장을 모두 이용하면 학습데이터의 크기가 매우 커지지만, 우리가 원하는 정보는 그리 많지 않기 때문입니다.

`encode` 함수에서 문장의 앞, 뒤에 window 만큼의 unknown vocab 을 추가합니다. 이는 context words 에 대한 padding 입니다. 이후 context_idxs 에서 단순히 list slicing 만 하여도 같은 크기의 input vector 를 만들 수 있습니다.

```python
    def encode(sent):
        idxs = [vocab_to_idx.get(w, n_vocabs) for w in sent]        
        idxs = [n_vocabs] * window + idxs + [n_vocabs] * window
        return idxs

    word_idxs = encode(sent)

    for i, word in enumerate(sent):
        # ...
        context_idxs = word_idxs[b:i+window] + word_idxs[i+window+1:e]
        context = np.hstack([wv_[idx] for idx in context_idxs])
```

In [ ]:
def create_dataset(vocab_to_idx, sents, seed_words, wv_, window=2, test_data=False):

    n_vocabs = len(vocab_to_idx)

    def contain_seed(sent):
        for word in sent:
            if word in seed_words:
                return True
        return False

    def encode(sent):
        idxs = [vocab_to_idx.get(w, n_vocabs) for w in sent]
        # padding
        idxs = [n_vocabs] * window + idxs + [n_vocabs] * window
        return idxs

    X = []
    words = []

    for sent in sents:

        if (not test_data) and (not contain_seed(sent)):
            continue

        n_words = len(sent)
        word_idxs = encode(sent)

        for i, word in enumerate(sent):
            if not (word in vocab_to_idx):
                continue

            b = i # i - window + window
            e = i + 2 * window + 1 # i + window + 1 + window

            context_idxs = word_idxs[b:i+window] + word_idxs[i+window+1:e]
            context = np.hstack([wv_[idx] for idx in context_idxs])
            X.append(context)
            words.append(word)

    X = np.vstack(X)
    Y = np.asarray([1 if w in seed_words else 0 for w in words], dtype=np.int)
    return X, Y, words

184,865 개의 학습 데이터가 만들어졌습니다.

In [ ]:
X, Y, words = create_dataset(vocab_to_idx, sents, seed_words, wv_)
X.shape, Y.shape

Scikit-learn 도 partial_fit 함수를 이용하면 minibatch style 로 구현할 수 있습니다. fit 함수는 모델을 처음 학습할 때 이용하며, partial_fit 은 한 번 학습된 모델을 추가로 학습할 때 이용합니다. 또한 아래처럼 이전에 만든 모델을 입력할 수 있도록 구현하면 이용하던 모델에 추가 학습도 가능합니다. 

Classifier 를 만들 때 `max_iter=1` 로 설정하면 minibatch 처럼 만들 수 있습니다. Loss 는 positive class 의 데이터는 negative class 의 확률, negative class 의 데이터는 positive class 의 확률입니다. 이들을 모두 더하여 epoch 마다 출력도 합니다.

```python
def minibatch_style(model=None, ... ):

    if model is None:
        model = MLPClassifier(hidden_layer_sizes=hidden_size, activation='relu', max_iter=1)
```

In [ ]:
import math
from sklearn.neural_network import MLPClassifier

def minibatch_style(vocab_to_idx, sents, seed_words, wv_, model=None,
    n_batch_sents=50000, hidden_size=(50,), epochs=20, verbose=True):

    n_sents = len(sents)
    n_batchs = math.ceil(n_sents / n_batch_sents)

    if model is None:
        model = MLPClassifier(hidden_layer_sizes=hidden_size, activation='relu', max_iter=1)

    for epoch in range(epochs):

        loss = 0
        n_instances = 0

        for batch in range(n_batchs):

            b = batch * n_batch_sents
            e = min((batch + 1) * n_batch_sents, n_sents)
            X, Y, words = create_dataset(vocab_to_idx, sents[b:e], seed_words, wv_)

            partial_fit = (epoch > 0) or (batch > 0)
            if partial_fit:
                model.partial_fit(X, Y)
            else:
                model.fit(X, Y)

            prob = model.predict_proba(X)
            loss += prob[np.where(Y == 1)[0], 0].sum()
            loss += prob[np.where(Y == 0)[0], 1].sum()
            n_instances += X.shape[0]

            if verbose:
                avg_loss = loss / n_instances
                print('\rtrain epoch = {} / {}, batch = {} / {}, loss = {}'.format(
                    epoch+1, epochs, batch+1, n_batchs, avg_loss), end='')
        if verbose:
            print()

    return model

model = minibatch_style(vocab_to_idx, sents, seed_words, wv_)

앞선 `day7_2` 의 튜토리얼처럼 minibatch 형식으로 prediction 을 합니다.

In [ ]:
def minibatch_predict(vocab_to_idx, sents, seed_words, wv_, model, n_batch_sents=50000):
    y_prob = []
    y_words = []
    n_sents = len(sents)
    n_batchs = math.ceil(n_sents / n_batch_sents)

    for batch in range(n_batchs):

        b = batch * n_batch_sents
        e = min((batch + 1) * n_batch_sents, n_sents)
        X, _, words = create_dataset(vocab_to_idx, sents[b:e], seed_words, wv_, test_data=True)

        y_prob.append(model.predict_proba(X))
        y_words += words

        print('\rbatch prediction {} / {}'.format(batch+1, n_batchs), end='')
    print('\rbatch prediction {0} / {0}'.format(n_batchs))

    y_prob = np.vstack(y_prob)[:,1]
    y_words=  np.asarray(y_words)
    return y_prob, y_words

politician_prob, politician_words = minibatch_predict(vocab_to_idx, sents, seed_words, wv_, model)

In [ ]:
politician_prob.shape

In [ ]:
from collections import Counter
from collections import defaultdict

def extract(y_prob, y_words, min_prob=0.6):
    # word count
    word_counter = Counter(y_words)

    # prediction count
    pred_pos = defaultdict(int)
    for row in np.where(y_prob >= min_prob)[0]:
        pred_pos[y_words[row]] += 1
    pred_pos = {word:(word_counter[word], pos/word_counter[word])
                for word, pos in pred_pos.items()}
    return pred_pos

정치인 혹은 직함의 이름을 추출하였습니다 (구글링 해보면 많은 경우 정치인임을 확인하실 수 있습니다).

In [ ]:
pred_pos = extract(politician_prob, politician_words, min_prob=0.6)

for word, (count, prob) in sorted(pred_pos.items(), key=lambda x:-x[1][1])[:300]:
    if word in seed_words or len(word) == 1:
        continue
    if count < 30:
        continue
    print('{} ({})\t{:.3f}'.format(word, count, prob))

Word2Vec 에서의 유사어 중 일부를 학습에 이용하지 않았었습니다. 이들 중 어떤 단어가 정치인으로 판단되었는지 확인도 해 봅니다.

In [ ]:
for query in '인수위원회 임종성 전두환 전북도지사 정우택 지우마 진선미 최연혜 충북경제자유구역청 충북도의회 테메르 통일부장관 호세프 황광 황주홍 후세인'.split():
    if not query in pred_pos:
        continue
    count, prob = pred_pos[query]
    print('{} ({})\t{:.3f}'.format(query, count, prob))

좀 더 저에게 익숙한 엔터테인 도메인에서 동일한 작업을 수행하였습니다.

In [ ]:
seed_words = {w for w, _ in word2vec.wv.most_similar('아이오아이', topn=20)}
seed_words = {w for w, _ in word2vec.wv.most_similar('트와이스', topn=20)}
seed_words.update({w for w, _ in word2vec.wv.most_similar('에이핑크', topn=20)})
seed_words.update({w for w, _ in word2vec.wv.most_similar('강호동', topn=20)})
seed_words = set(
    '''강호동 경리 고복실 김국진 김규종 다이아 동방신기 듀오 레이디 몬스타엑스 바스타즈
    박수홍 박재범 블락비 비스트 빅뱅 샤이니 세븐 손나은 신용재 신화 아이오아이 에이핑크
    엑소 오블리스 우태운 원호 위너 이경규 이별 이화신 제작진 종이학 종현 진석 치타 컴백
    키스신 타이틀곡 태민 트와이스 표나리 피오 형님'''.split())

앞서 만들어둔 함수를 재활용 할 수 있습니다.

In [ ]:
model = minibatch_style(vocab_to_idx, sents, seed_words, wv_)
entertainer_prob, entertainer_words = minibatch_predict(vocab_to_idx, sents, seed_words, wv_, model)

회사 이름이나 정치 용어가 섞여있긴 하지만 많은 단어들이 엔터테이너 혹은 엔터테인 관련 단어입니다. 그리고 이러한 window classification 기반 학습 방법은 positive class 의 단어들이 주로 등장했던 문맥을 그대로 외운 뒤, 동일한 문맥에 등장한 단어를 탐색합니다. 그렇기 때문에 토픽이 조금 다르다면 동일한 문맥이 없을 수도 있습니다.

In [ ]:
pred_pos = extract(entertainer_prob, entertainer_words, min_prob=0.85)

for word, (count, prob) in sorted(pred_pos.items(), key=lambda x:-x[1][1])[:1000]:
    if word in seed_words or len(word) == 1:
        continue
    if count < 50:
        continue
    print('{} ({})\t{:.3f}'.format(word, count, prob))